---
title: Exploratory data analysis 
description: "Exploring data using univariate, bivariate and multivariate analysis"
author: "Jeevith Hegde"
date: 2025-08-09 12:02 +0200
format: 
    html
toc: true
lang: en
jupyter: python3
ipynb-shell-interactivity: all
execute:
  echo: false
categories: ["datascience", "fundamentals"]
tags: ["notes", "eda"]

comments:
  giscus:
    repo: jeev20/jeev20.github.io
---





# What is in my data? 


According to [Campusx-YouTube](https://youtu.be/mJlRTUuVr04?si=RCSLJGWOIAnQZhvW) these are some of the suggested questions to answer when a new dataset is obtained.
```{mermaid}
mindmap
  root((Quick Data Exploration))
    1 - How big is the data? 
    2 - How does the data look like?
    3 - What type of datatypes are found in the data?
    4 - Are there missing data in this dataset?
    5 - How does the data look mathematically?
    6 - Are there duplicate rows in this data?
    7 - What is the correlation between the columns in this data?
            
```

## Importing modules

In [ ]:
# | echo: true
import duckdb
import polars as pl
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

## Reading data

In [ ]:
# | echo: true
# ts is in epoch time format so converting it to timestamp
# rounding values for temperature and humidity
# converting temperature from farhaneit to celsius

input_data_path = f"../data/iot/iot_telemetry_data.parquet"


df_raw = duckdb.sql(
    f"SELECT ts, to_timestamp(ts) AS timestamp, device, temp,ROUND((temp - 32) * 5.0 / 9, 4) AS temp_c, ROUND(humidity, 4) AS humidity, lpg, smoke, light FROM '{input_data_path}'"
)

## Exploring the data
The seven questions to get insight into the data 

### How big is the data? 

In [ ]:
# | echo: true
# Converting to polars to easy statistics and exploration
df_pl = df_raw.pl()  
df_pl.shape

### How does the data look like?

In [ ]:
# | echo: true
df_pl.head()

### What type of datatypes are found in the data?

In [ ]:
# | echo: true
duckdb.sql("SUMMARIZE df_raw;")

### Are there missing data in this dataset?
This dataset is quiet clean, there are no missing data in the input data in any feature. [Docs Reference](https://docs.pola.rs/user-guide/expressions/missing-data/)

In [ ]:
#| echo: true
df_pl.null_count()

### How does the data look mathematically?

In [ ]:
#| echo: true
df_pl.describe()

### Are there duplicate rows in this data?
Although the below script shows there are duplicates, it looks like polars and pandas work differently here so it is important to check both. While polars returns all occurances of the duplicate, pandas only gets the duplicated values. Hence polars shows the shape to be `(26,8)` while pandas returns `13` duplicated rows in [this dataset](https://www.kaggle.com/datasets/garystafford/environmental-sensor-data-132k)

In [ ]:
#| echo: true
dup_mask = df_pl.is_duplicated()
duplicates = df_pl.filter(dup_mask)  # Filter to show duplicated rows only

df_pd = df_pl.to_pandas().duplicated().sum() # from pandas
df_pd
duplicates  # from polars

### What is the correlation between the columns in this data?

We need to use pandas here as the `.corr()` function in pandas provides a more readable table for inspecting correlation

In [ ]:
#| echo: true
# Select only numeric columns for the correlation matrix
numeric_df = df_pl.select(pl.col(pl.Float64 or pl.Int64)) 
numeric_df.to_pandas().corr(method='pearson')
numeric_df.to_pandas().corr(method='spearman')
numeric_df.to_pandas().corr(method='kendall')

# Exploratory data analysis (EDA)

```{mermaid}
mindmap
  root((EDA))
    **Univariate data analysis** <br>Independent analysis of individual columns of the dataset. 
    **Bivariate data analysis** <br> Pairwise analysis of two columns of the dataset. 
    **Multivariate data analysis**<br> Simultaneously analysis of two or more columns.
            
```


## Univariate data analysis
### Handling categorial data

#### Countplot
Count plots provide the unique values of the column and their frequency of occurance. This provides a understanding on how the data is spread across the classes / categories of the

In [ ]:
#| echo: true
sn.countplot(df_pl.select(pl.col('device')), x='device')

#### PieChart
A PieChart can be customized to show both the count of categories and their percentages in the dataset. 

In [ ]:
#| echo: true
vc = df_pl.to_pandas()['device'].value_counts()
vc.plot(kind='pie', autopct=lambda pct: f'{int(round(pct * vc.sum() / 100))}\n({pct:.1f}%)')

### Handling numerical data

#### Histogram
Histogram is a way to visualize a frequency table. The datapoints can be binned to `n` number of bins. 

In [ ]:
#| echo: true
sn.histplot(df_pl, x='humidity', bins=60)

In [ ]:
#| echo: true
sn.histplot(df_pl, x='temp_c', bins=60)

#### Distplot
Density plots show the distribution of the data values as a continuous line. It is a smoothed version of a histogram and is calculated via a `kernel density estimate` (page 24 - Practical Statistics for Data Scientists). The main difference between an histogram and a distplot is the `y-axis` scale. Displot uses probability scale and histogram uses frequency / count of observations. 


In [ ]:
#| echo: true
sn.displot(df_pl, x='humidity', kind="kde")

In [ ]:
#| echo: true
sn.displot(df_pl, x='temp_c', kind="kde")

But remember our data has three different sensors. Would be great to ensure we can see the probablity density function of each one seperately. But this will fall in Bivariate data analysis, as we are using another column to make sense of the `humidity` or `temp_c` column.

In [ ]:
#| echo: true
sn.kdeplot(df_pl, x='humidity', hue='device')

#### Boxplot 
Boxplot give a quick way to visualize the distribution of data and provide boundaries to the observation outside which we find the `outliers`. It also shows the `25th percentile`, `median`, `75th percentile`. 

The visual also helps quickly notice the outliers in the observations on both lower and higher boundries. 

This is also called as a `5 point summary` visual.

![BoxplotVisualized](images/Boxplot.jpg)

In [ ]:
#| echo: true
sn.boxplot(df_pl, x='humidity')

In [ ]:
#| echo: true
sn.boxplot(df_pl, x='temp_c')

## Bivariate data analysis
### Scatterplot (Numerical to Numerical)

In [ ]:
#| echo: true
sn.scatterplot(df_pl, x='temp_c', y='humidity' )

### Barplot (Numerical to categorical)

In [ ]:
#| echo: true
sn.barplot(df_pl, x='temp_c', y='device')

### Boxplot (Numerical to categorical)

In [ ]:
#| echo: true
sn.boxplot(df_pl, x='temp_c', y='device')

## Multivariate data analysis


In [ ]:
#| echo: true
sn.scatterplot(df_pl, x='temp_c', y='humidity', hue='device' )

In [ ]:
#| echo: true

# Convert Polars DataFrame to pandas for seaborn plotting
df = df_pl.to_pandas()

# Get unique devices (you want 3 scatterplots, one for each device)
devices = df['device'].unique()

# Create a figure with 1 row and 3 columns for side-by-side plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharex=True, sharey=True)

# Loop through devices and axes to plot each device's scatterplot
for ax, device in zip(axes, devices):
    subset = df[df['device'] == device]
    sn.scatterplot(data=subset, x='temp_c', y='humidity', ax=ax)
    ax.set_title(f'Device: {device}')
    ax.set_xlabel('Temperature (°C)')
    ax.set_ylabel('Humidity')

plt.tight_layout()
plt.show()

# Automated EDA using pandas-profiling
This is a quick way to perform EDA and study the data. However, this only works on pandas dataframe. So if we have very large datasets, this approach may or may not be performant. 

That said, this approach eliminates most of the hassle in understanding the input data. 

It lacks in some visualization, which need to be performed to get a better idea of the data. 

In [ ]:
#| echo: true
from ydata_profiling import ProfileReport
profile = ProfileReport(df_pl.to_pandas(), title="Data Profiling Report")
profile.to_notebook_iframe()